In [4]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [5]:
hist_data = pd.read_csv("histogram_data.csv")

In [6]:
def createIndex(es,indexName):
    mapping={
  "mappings": {
    "_doc": {
      "properties": {
        "rssi_percs_25": {
          "type": "long"
        },
        "rxWeightedPhyRate": {
          "type": "long"
        },
        "txWeightedPhyRate": {
          "type": "long"
        }
      }
    }
  }
}
    
    created=es.indices.create(index=indexName, body=mapping, include_type_name=True)
    
    return created


In [7]:
def ingestData(datadict,es,indexName,doc_type):
    global count_Path
    documents = []
    for row in datadict:
        info = {}
        info["_index"] = indexName
        info["_type"] = doc_type
        info["_source"] = row
        doc = info
        documents.append(doc)
    try:
        helpers_response=helpers.bulk(es, documents,raise_on_error=False)
        print(helpers_response)
    except:
        print("***error chunk****")

In [9]:
index_name = 'prodapt_hist_data'
es = Elasticsearch([{'host': 'localhost', 'port': 9200,'timeout': 3000}])
index = index_name
createIndex(es,index)
data_dict = hist_data.T.to_dict().values()
res = [ele for ele in ({key: val for key, val in sub.items() if val} for sub in data_dict) if ele]
ingestData(res,es,index_name,'_doc')

(21, [])
